# Labsheet 2: Multi-tasking with millis()

We will explore the built in Arduino function _millis()_ to allow us to imitate a task scheduling system for running blocks of code.  This is useful if we want to do multiple things at once.  For instance, we may wish to update our motor control every 0.5 seconds, whilst updating our line sensor every 0.1 seconds, and to make sure these happen independently.  

The _millis()_ function reports the time passed in milliseconds since the arduino was powered up.  We can use this to decide to run a block of code once a period of time in milliseconds has elapsed.  If you wanted a smaller time interval, there is also a _micros()_ function which operates in the same way we use _millis()_, but reports microseconds since power-up.

For _millis()_ to keep track of time, the arduino has some code running in the background by default.  Behind _millis()_, a peripheral device called a **timer** is independently tracking the passage of time, and running a piece of code called an **interrupt service routine (ISR)** which updates the count for _millis()_.  

Using the built in function _millis()_ means that we do not have to get involved with using a **timer/ISR**.  After this worksheet, you may wish to look at the advanced labsheet on Interrupts & Timers.  The advanced labsheet isn't necessary to complete the assessment.  If you want to work with microcontrollers in the future it is worthwhile investigating.  




# Exercise 1: Using Arduino _millis()_ to Multi-task

When you have written code before, you may have used the delay() function or delayMicroseconds() function to get your program to stop and wait.  For instance, you may have read a sensor (or blink an LED) every 10 seconds with code like the following:

<pre>
<font color="#434f54">&#47;&#47; Remember: loop is called again and again.</font>
<font color="#00979c">void</font> <font color="#5e6d03">loop</font><font color="#000000">(</font><font color="#000000">)</font> <font color="#000000">{</font>
 &nbsp;
 &nbsp;&nbsp;&nbsp;<font color="#00979c">int</font> <font color="#000000">sensor_value</font> <font color="#434f54">=</font> <font color="#d35400">analogRead</font><font color="#000000">(</font> <font color="#000000">A0</font> <font color="#000000">)</font><font color="#000000">;</font> &nbsp;&nbsp;&nbsp;<font color="#434f54">&#47;&#47; Do a sensor read, analog pin 0</font>

 &nbsp;&nbsp;&nbsp;<font color="#d35400">delay</font><font color="#000000">(</font> <font color="#000000">10000</font> <font color="#000000">)</font><font color="#000000">;</font> &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<font color="#434f54">&#47;&#47; loop() is stuck waiting here for 10seconds.</font>
 &nbsp;&nbsp;&nbsp;
<font color="#000000">}</font>

</pre>

The problem with the above code is that when _delay( 10000 )_ is called, your program cannot do anything else - it will sit waiting for 10000 milliseconds.  Generally, this means your robot would not be able to respond to any new information.  This is referred to as **blocking** code.  

Robots typically have to do many things at once, so it is no good if our program becomes blocked.  Instead, we want to write **non-blocking** code - code that can run when it is necessary, and otherwise let the microcontroller CPU do other tasks.  Blocking and non-blocking refer to the fact that code is **sequential**, the program runs (generally speaking) from top to bottom without any real intelligence for itself.

The arduino _millis()_ function provides how many milliseconds have passed since the arduino was turned on (a timestamp).  The Arduino is configured by default to alway count up in milliseconds as a background task.  How this actually works is that the Timer peripheral _timer0_ is configured to keep track of time for you by using an _interrupt service routine (ISR)_.   It is useful to remember that _timer0_ is always counting up for you in the background from the moment the power is switched on.

_millis()_ provides the number of milliseconds elapsed in the format **_unsigned long_** (more info <a href="https://www.arduino.cc/reference/en/language/variables/data-types/unsignedlong/">here</a>).  This means the Arduino can count up to 4,294,967,295 milliseconds.  That is approximately 50 days.  After 50 days, it will reach the maximum value and begin counting from 0 again, which is called an **overflow**.  This means we can only use _millis()_ as a reliable mechanism for 50 days - it is not a perfect solution.  However, your Romi will probably run out of battery power before _millis()_ finished a complete count cycle.  

We can use _millis()_ to keep track of how much time has passed by recording the millisecond count from one time to another - we will refer to these as **timestamps**.  Note that _millis()_ is relative time - how much time has passed, **not the actual time of day**.  We therefore compare two values reported by _millis()_ to work out how much time has passed.  

<img src="timestamps.png">


By encapsulating a block of code within an *if...* statement, we can ensure it only operates when a certain amount of time has elapsed.  Note that your code is still _sequential_, and we are making a clever use of _selection_.

The following code provides a complete working example:

<pre>
<font color="#00979c">unsigned</font> <font color="#00979c">long</font> <font color="#000000">last_timestamp</font><font color="#000000">;</font> &nbsp;&nbsp;<font color="#434f54">&#47;&#47; We will store a timestamp in this.</font>
<font color="#00979c">int</font> <font color="#000000">sensor_value</font><font color="#000000">;</font> &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<font color="#434f54">&#47;&#47; We will store a sensor reading in this.</font>

<font color="#434f54">&#47;&#47; Remember: Setup only runs once when the arduino is powered up.</font>
<font color="#00979c">void</font> <font color="#5e6d03">setup</font><font color="#000000">(</font><font color="#000000">)</font> <font color="#000000">{</font>

 &nbsp;<font color="#d35400">pinMode</font><font color="#000000">(</font><font color="#000000">A0</font><font color="#434f54">,</font> <font color="#00979c">INPUT</font> <font color="#000000">)</font><font color="#000000">;</font> &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<font color="#434f54">&#47;&#47; Setup up A0 as input to read.</font>

 &nbsp;<font color="#000000">last_timestamp</font> <font color="#434f54">=</font> <font color="#d35400">millis</font><font color="#000000">(</font><font color="#000000">)</font><font color="#000000">;</font> &nbsp;&nbsp;&nbsp;<font color="#434f54">&#47;&#47; We set an intial timestamp value.</font>


<font color="#000000">}</font>


<font color="#434f54">&#47;&#47; Remember: loop is called again and again.</font>
<font color="#00979c">void</font> <font color="#5e6d03">loop</font><font color="#000000">(</font><font color="#000000">)</font> <font color="#000000">{</font>

 &nbsp;&nbsp;&nbsp;<font color="#434f54">&#47;&#47; Get how much time has passed right now.</font>
 &nbsp;&nbsp;&nbsp;<font color="#00979c">unsigned</font> <font color="#00979c">long</font> <font color="#000000">time_now</font> <font color="#434f54">=</font> <font color="#d35400">millis</font><font color="#000000">(</font><font color="#000000">)</font><font color="#000000">;</font> &nbsp;&nbsp;&nbsp;&nbsp;

 &nbsp;&nbsp;&nbsp;<font color="#434f54">&#47;&#47; Work out how many milliseconds have gone passed by subtracting</font>
 &nbsp;&nbsp;&nbsp;<font color="#434f54">&#47;&#47; our two timestamps. &nbsp;time_now will always be bigger than the</font>
 &nbsp;&nbsp;&nbsp;<font color="#434f54">&#47;&#47; time_of_read (except when millis() overflows after 50 days).</font>
 &nbsp;&nbsp;&nbsp;<font color="#00979c">unsigned</font> <font color="#00979c">long</font> <font color="#000000">elapsed_time</font> <font color="#434f54">=</font> <font color="#000000">time_now</font> <font color="#434f54">-</font> <font color="#000000">last_timestamp</font><font color="#000000">;</font>



 &nbsp;&nbsp;&nbsp;<font color="#434f54">&#47;&#47; See if 10000 milliseconds have ellapsed</font>
 &nbsp;&nbsp;&nbsp;<font color="#434f54">&#47;&#47; If not, this block is skipped.</font>
 &nbsp;&nbsp;&nbsp;<font color="#5e6d03">if</font><font color="#000000">(</font> <font color="#000000">elapsed_time</font> <font color="#434f54">&gt;</font> <font color="#000000">10000</font> <font color="#000000">)</font> <font color="#000000">{</font>

 &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<font color="#434f54">&#47;&#47; Since 10000ms elapsed, we overwrite our last_timestamp with the current time</font>
 &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<font color="#434f54">&#47;&#47; so that another 10000ms is needed to pass.</font>
 &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<font color="#000000">last_timestamp</font> <font color="#434f54">=</font> <font color="#d35400">millis</font><font color="#000000">(</font><font color="#000000">)</font><font color="#000000">;</font>

 &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<font color="#434f54">&#47;&#47; Do our sensor read.</font>
 &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<font color="#000000">sensor_value</font> <font color="#434f54">=</font> <font color="#d35400">analogRead</font><font color="#000000">(</font> <font color="#000000">A0</font> <font color="#000000">)</font><font color="#000000">;</font>

 &nbsp;&nbsp;&nbsp;<font color="#000000">}</font>

 &nbsp;&nbsp;&nbsp;<font color="#434f54">&#47;&#47; Code outside the above if{} will run on every loop!</font>
 &nbsp;&nbsp;&nbsp;<font color="#434f54">&#47;&#47; Therefore code here is no longer stopped waiting for delay()</font>

<font color="#000000">}</font>

</pre>


- Extend the example code to include multiple *if...* blocks  to flash the three LED's on the Romi independently at different time intervals.  
    - Change the pinMode() call to set the three LED pins to outputs.
    - Change the analogRead() call to a digitalWrite() for your LED pins.
    - **Hint:** You can find the code to setup and flash an LED (a digital pin) within **Lab Sheet 1: Meet the Romi**. 
    - **Hint:** you will need extra _unsigned long_ timestamps variables declared globally to keep track activaty different _if..._ statement blocks.  Give them meaningful names (e.g. unsigned long red_led_ts;).  It is common to use global variables with microcontrollers, and when you compile your code you should be able to see exactly how much system memory you code will use.
    - **Hint:**  You can find which pins the 3 LED's are on using the table <a href="https://www.pololu.com/docs/0J69/3.8">here</a>.  For the pin number to use in your code, look in the column labelled 'Arduino Pin Names'.  
    - **Note:** You may notice that the LED labelled 13 is **on** when the LED's labelled TX & RX are **off**, and vice-versa.  This is because the electronics are wired in opposite ways for these LED's.  
    - **Note:** If you use the **Serial()** functions , this will operate the TX and RX LED's, making them blink out of your control.  It is common for many devices to share pins on a microcontroller.
    
    

# Exercise 2: Using the Buzzer & millis() or micros()

In this exercise we'll configure the Romi to activate the audio buzzer.  In labsheet 1 we used _analogWrite()_ to activate the buzzer.  _analogWrite()_ creates a voltage level by switching a digital pin on and off very fast - called pulse width modulation (PWM).  With analogWrite(), we effectively control the voltage level, but not the frequency (frequency is roughly how fast we are turning it on and off).  Therefore, your buzzer could only beep with one tone, but you were controlling the volume (the power).  

The frequency at which the buzzer is turned on-and-off will determine the pitch of sound you hear.  We can write our own PWM functionality by using _digitalWrite()_ to switch between on and off (1/0), and we can use _millis()_ or _micros()_  to control the frequency.  

**Note that** it is very easy to create tones that the human ear cannot detect, or that we will hear as strange noises.  Whilst working on this labsheet, cats, dogs, bats and other animals may decide they do not like you.

The following code provides a skeleton template for you to generate sounds of different pitch:


<pre>

<font color="#434f54">&#47;&#47; Setting a #define is like a find-and-replace</font>
<font color="#5e6d03">#define</font> <font color="#000000">BUZZER_PIN</font> <font color="#000000">????</font>

<font color="#434f54">&#47;&#47; Global variables.</font>


<font color="#434f54">&#47;&#47; Remember, setup runs once when the romi is powered up</font>
<font color="#00979c">void</font> <font color="#5e6d03">setup</font><font color="#000000">(</font><font color="#000000">)</font> <font color="#000000">{</font>
 &nbsp;

 &nbsp;<font color="#434f54">&#47;&#47;Start a serial connection</font>
 &nbsp;<b><font color="#d35400">Serial</font></b><font color="#434f54">.</font><font color="#d35400">begin</font><font color="#000000">(</font><font color="#000000">BAUD_RATE</font><font color="#000000">)</font><font color="#000000">;</font>
&nbsp; <font color="#434f54">&#47;&#47; Wait for stable connection, report reset.</font>
&nbsp; <font color="#d35400">delay</font><font color="#000000">(</font><font color="#000000">1000</font><font color="#000000">)</font><font color="#000000">;</font>
&nbsp; <b><font color="#d35400">Serial</font></b><font color="#434f54">.</font><font color="#d35400">println</font><font color="#000000">(</font><font color="#005c5f">&#34;***RESET***&#34;</font><font color="#000000">)</font><font color="#000000">;</font>
<br>
 &nbsp;<font color="#434f54">&#47;&#47; Set pin 6 (buzzer) to output.</font>
 &nbsp;<font color="#d35400">pinMode</font><font color="#000000">(</font> <font color="#000000">BUZZER_PIN</font><font color="#434f54">,</font> <font color="#00979c">OUTPUT</font><font color="#000000">)</font><font color="#000000">;</font>

 &nbsp;<font color="#434f54">&#47;&#47; Initialise your global variables.</font>

<font color="#000000">}</font>

<font color="#434f54">&#47;&#47; Remember, loop repeats again and again.</font>
<font color="#00979c">void</font> <font color="#5e6d03">loop</font><font color="#000000">(</font><font color="#000000">)</font> <font color="#000000">{</font>


 &nbsp;<font color="#434f54">&#47;&#47; Implement a millis() or micros() task block</font>
 &nbsp;<font color="#434f54">&#47;&#47; to toggle the state of the buzzer.</font>

<font color="#000000">}</font>
</pre>


- <a href="https://www.pololu.com/docs/0J69/3.8">Find out which pin the piezo buzzer is attached to</a>, add this to the above code.

- Refering to the previous exercise, implement a _millis()_ task block to toggle the state of your buzzer.  Avoid using an iterator (like for(), while()) within _loop()_.
- Explore values of buzz_interval to find different sounds.
- Update the code to use micros() rather than millis() to achieve more pitches of sound.
- **Hint**: you'll need to create some global variables to remember the state and timestamp for your buzzer routine between _loop()_ executions.
- To stop your course mates going mad, you may wish to stick a piece of bluetack, plastacine, or tape over the hole of the buzzer on your Romi to make it quieter.  To locate the buzzer on your Romi, locate the part making the annoying noises.


# Exercise 3: 


- Write an _if()_ statement into your loop so that the the value of buzz_interval increments every 1000 milliseconds.  Done correctly, you should hear the beeping going up in pitch every second.
- **Hint:** You can use another timestamp variable and _if()_ block to run every 1000 milliseconds, seperate from your buzzer updating block.  Essentially, running two tasks at two different time intervals.

    
# Exercise 4:

- Use an array to create a list of specific time interval values for your buzzer, selected by you.  Essentially, a list of different buzzing tones.  Update your code to move through and refer to this array.  Move through your array at 1 second intervals.  
- See if you can make your Romi play a tune you recognise.
- **Hint**: Remember to check for indexing beyond the length of your array.
- **Hint**: If you've not used arrays in C before, here is some example syntax:


<pre>

<font color="#434f54">&#47;&#47; Global scope</font>
<font color="#00979c">unsigned</font> <font color="#00979c">long</font> <font color="#000000">intervals</font><font color="#000000">[</font><font color="#000000">6</font><font color="#000000">]</font> <font color="#434f54">=</font> <font color="#000000">{</font> <font color="#000000">1000</font><font color="#434f54">,</font> <font color="#000000">500</font><font color="#434f54">,</font> <font color="#000000">100</font><font color="#434f54">,</font> <font color="#000000">600</font><font color="#434f54">,</font> <font color="#000000">200</font><font color="#434f54">,</font> <font color="#000000">500</font> <font color="#000000">}</font><font color="#000000">;</font>
<font color="#00979c">int</font> <font color="#000000">interval_index</font><font color="#000000">;</font>

<font color="#434f54">&#47;&#47; ... elsewhere in code</font>

 &nbsp;<font color="#000000">interval_index</font> <font color="#434f54">=</font> <font color="#000000">0</font><font color="#000000">;</font> &nbsp;&nbsp;<font color="#434f54">&#47;&#47; here, using position 0 in array.</font>
 &nbsp;<font color="#000000">buzz_interval</font> <font color="#434f54">=</font> <font color="#000000">intervals</font><font color="#000000">[</font> <font color="#000000">interval_index</font> <font color="#000000">]</font><font color="#000000">;</font> &nbsp;&nbsp;&nbsp;<font color="#434f54">&#47;&#47; read array position.</font>

<font color="#434f54">&#47;&#47; ... code continues.</font>

</pre>

# Exercise 5:

- Take the code you have experimented with in this Labsheet and see if you can encapsulate it within a function.  The aim being, that you can make a call to your function to play a tune.
- See if you can implement your function without the use of iteration (such as a for() or while() loop), so that your _loop()_ execution is not blocked.
- Just for fun, you may want to make this your 'Victory Music', which your Romi will be able to play when you successfully complete the Line Following Challenge.